In [1]:
from pathlib import Path
from rasterio.plot import reshape_as_raster, reshape_as_image

import matplotlib.pyplot as plt
import numpy as np
import geopandas as gpd
import pandas as pd
import sys
import rasterio as rio

/home/nilscp/.pyenv/versions/3.8.0/envs/computer-vision/lib/python3.8/site-packages/geopandas/_compat.py:106: UserWarning: The Shapely GEOS version (3.8.0-CAPI-1.13.1 ) is incompatible with the GEOS version PyGEOS was compiled with (3.10.1-CAPI-1.16.0). Conversions between both will be slow.
  warnings.warn(


In [2]:
sys.path.append("/home/nilscp/GIT/crater_morphometry/rim_detection")
sys.path.append("/home/nilscp/GIT/crater_morphometry/preprocessing")
sys.path.append("/home/nilscp/GIT/crater_morphometry")

import rim_detection
import geomorphometry

In [14]:
df = gpd.read_file("/home/nilscp/GIT/crater_morphometry/data/rayed_craters/rayed_craters.shp")
dfa = df.copyy()

In [15]:
dfa.crs = 'PROJCS["Moon_Equidistant_Cylindrical",GEOGCS["Moon 2000",DATUM["D_Moon_2000",SPHEROID["Moon_2000_IAU_IAG",1737400.0,0.0]],PRIMEM["Greenwich",0],UNIT["Decimal_Degree",0.0174532925199433]],PROJECTION["Equidistant_Cylindrical"],PARAMETER["False_Easting",0],PARAMETER["False_Northing",0],PARAMETER["Central_Meridian",-125],PARAMETER["Standard_Parallel_1",78],UNIT["Meter",1]]'

In [16]:
dfa.crs.to_proj4()

/home/nilscp/.pyenv/versions/3.8.0/envs/computer-vision/lib/python3.8/site-packages/pyproj/crs/crs.py:1256: UserWarning: You will likely lose important projection information when converting to a PROJ string from another format. See: https://proj.org/faq.html#what-is-the-best-format-for-describing-coordinate-reference-systems
  return self._crs.to_proj4(version=version)


'+proj=eqc +lat_ts=78 +lat_0=0 +lon_0=-125 +x_0=0 +y_0=0 +R=1737400 +units=m +no_defs +type=crs'

In [18]:
location_of_craters = '/home/nilscp/GIT/crater_morphometry/data/rayed_craters/rayed_craters_centroids.shp'
gdf = gpd.read_file(location_of_craters)

In [20]:
gdf.head(50)

,lon,lat,tag,CRATER_ID,MED_DIAM,UNC_DIAM,DIAM_25,DIAM_75,DIAM_MIN,DIAM_MAX,...,NO_CRAT,LIGHT_CRA,MED_CRA,HEAV_CRA,TOTAL,CLEM,Shape_Leng,Shape_Area,diam,geometry
0,102.881037,35.967984,standard,crater0000,22473.800,220.1080,21240.200,24021.30,20079.400,57037.80,...,-1.0,0.0,0.0,0.0,-5.0,1.0,80237.333599,5.038171e+08,22785.584284,POINT (3119697.738 1090676.548)
1,176.328293,28.660823,standard,crater0001,20386.700,183.6790,19444.500,21731.50,18044.000,41313.10,...,0.0,0.0,0.0,0.0,-2.0,3.0,71002.954102,3.986148e+08,21103.335712,POINT (5346864.617 869096.562)
2,-174.967708,37.294420,standard,crater0002,27439.100,258.5130,24955.000,28780.80,19280.900,64148.00,...,0.0,0.0,0.0,0.0,-3.0,1.0,85956.429109,5.765708e+08,24166.395788,POINT (-5305607.122 1130899.760)
3,-108.089525,44.389105,standard,crater0003,26030.800,39.1647,25594.400,26626.60,24126.500,28428.70,...,0.0,0.0,0.0,2.0,2.0,3.0,102971.791274,8.091061e+08,27132.614904,POINT (-3277636.557 1346039.343)
4,-93.290796,27.943836,standard,crater0004,28767.900,176.1720,28137.900,29329.00,26416.300,61426.00,...,0.0,0.0,0.0,2.0,2.0,3.0,95938.629290,7.282259e+08,28620.039188,POINT (-2828889.486 847358.533)
5,-67.726198,31.834792,standard,crater0005,19587.500,39.2335,19105.900,20015.40,18019.300,22439.80,...,0.0,0.0,0.0,2.0,1.0,4.0,69195.708175,3.772411e+08,20200.721231,POINT (-2053685.245 965342.101)
6,-70.904629,28.159517,standard,crater0006,24490.300,38.4028,23848.300,24843.60,22750.600,27092.80,...,0.0,0.0,0.0,2.0,1.0,4.0,82431.527623,5.375075e+08,24563.720190,POINT (-2150065.923 853896.702)
7,-2.452706,6.718365,standard,crater0007,18168.000,26.2082,17827.400,18563.60,17249.300,19851.90,...,0.0,0.0,0.0,2.0,2.0,3.0,59486.700524,2.815861e+08,18869.781366,POINT (-74374.278 203724.104)
8,-18.770805,0.972621,standard,crater0008,11810.600,16.2250,11607.800,12028.50,10957.400,12866.80,...,0.0,0.0,0.0,2.0,0.0,1.0,37625.202138,1.126515e+08,11975.532406,POINT (-569193.702 29493.157)
9,-100.244616,3.302646,standard,crater0009,9476.560,43.3393,9205.260,9888.58,7681.960,17397.10,...,-1.0,0.0,0.0,0.0,-5.0,1.0,30772.426127,7.535329e+07,9786.961470,POINT (-3039752.633 100147.379)


In [7]:
len(list(np.arange(512)))

512

In [17]:
df.crs.to_proj4()

'+proj=eqc +lat_ts=0 +lat_0=0 +lon_0=0 +x_0=0 +y_0=0 +R=1737400 +units=m +no_defs +type=crs'

In [9]:
longitude = -172
central_meridian = 'PARAMETER["Central_Meridian",180]'
central_meridian = central_meridian.replace('180',str(int(longitude)))
central_meridian

'PARAMETER["Central_Meridian",-172]'

In [4]:
# some information related to the crater008
crater_id = 'crater0173'
i = 173

crater_dem = Path('/home/nilscp/tmp/crater/' + crater_id + '.tif')
scaling_factor = 0.5
dem_resolution = 59.22529380000000288
crater_radius = (df.Diam_km.iloc[i]*1000.0) / 2.0

In [5]:
(xnewcenter, ynewcenter, rnew, residu) = rim_detection.first_run(crater_dem, crater_radius, scaling_factor)

/home/nilscp/GIT/crater_morphometry/rim_detection/rim_detection.py:519: FutureWarning: `rcond` parameter will change to the default of machine precision times ``max(M, N)`` where M and N are the input matrix dimensions.
To use the future default and silence this warning we advise to pass `rcond=None`, to keep using the old, explicitly pass `rcond=-1`.
  be4stat = np.linalg.lstsq(sbe4,zbe4)[0]
/home/nilscp/GIT/crater_morphometry/rim_detection/rim_detection.py:520: FutureWarning: `rcond` parameter will change to the default of machine precision times ``max(M, N)`` where M and N are the input matrix dimensions.
To use the future default and silence this warning we advise to pass `rcond=None`, to keep using the old, explicitly pass `rcond=-1`.
  aftstat = np.linalg.lstsq(saft,zaft)[0]
/home/nilscp/.pyenv/versions/3.8.0/envs/computer-vision/lib/python3.8/site-packages/numpy/core/fromnumeric.py:3419: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
/hom

9092.359215048382 9085.224792723637
